In [3]:
import rasterio as rio
from rasterio.crs import CRS
from affine import Affine

from netCDF4 import Dataset
import os
import sys
import logging
logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)

In [4]:
DATA_DIR = '/Users/nathansuberi/Documents/GitHub/ResourceWatchCode/static-data-connectors/soc_073_gridded_gdp/data/'
GS_PREFIX = 'soc_073_gridded_gdp'
EE_COLLECTION = 'soc_073_gridded_gdp'
FILENAME = 'soc_073_{indicator}_{resolution}_{year}'

gdp_percap_PPP = {'dataset':'GDP_per_capita_PPP_1990_2015_v2.nc',
                    'name':'gdp_percap_ppp',
                    'resolution':'5arcmin'}
gdp_PPP = {'dataset':'GDP_PPP_1990_2015_5arcmin_v2.nc',
                    'name':'gdp_ppp',
                    'resolution':'5arcmin'}
gdp_PPP_30arcsec = {'dataset':'GDP_PPP_30arcsec_v2.nc',
                    'name':'gdp_ppp',
                    'resolution':'30arcsec'}

data = [gdp_percap_PPP, gdp_PPP, gdp_PPP_30arcsec]
data = [gdp_PPP_30arcsec]

In [5]:
def getAssetName(indicator, resolution, year):
    '''get asset name from datestamp'''
    return os.path.join(EE_COLLECTION, FILENAME.format(indicator=indicator,resolution=resolution,year=year))

In [6]:
for ds in data:
    dataset = ds['dataset']
    name = ds['name']
    resolution = ds['resolution']

    logging.info('processing: ' + dataset)

INFO:root:processing: GDP_PPP_30arcsec_v2.nc


In [7]:
for ds in data:
    dataset = ds['dataset']
    name = ds['name']
    resolution = ds['resolution']

    logging.warning('processing: ' + dataset)
    logging.warning(DATA_DIR + dataset)

    # Extract data
    nc = Dataset(DATA_DIR + dataset)
    var = list(nc.variables.keys())[-1]
    logging.warning('variable: ' + var)
    nc_data = nc[var]

    # Set info for new tif
    nodata = nc_data.getncattr("missing_value")
    shape = nc_data.shape
    logging.warning(shape)
    bands = shape[0]
    if ds == gdp_PPP_30arcsec:
        bandNames = ['1990','2000','2015']
    height = shape[1]
    width = shape[2]

    # Return lat info
    south_lat = -90
    north_lat = 90

    # Return lon info
    west_lon = -180
    east_lon = 180
    # Transformation function
    transform = rio.transform.from_bounds(west_lon, south_lat, east_lon, north_lat, width, height)

    profile = {
        'height':height,
        'width':width,
        'count':1,
        'transform':transform,
        'crs':CRS({'init': 'EPSG:4326'}),
        'driver':'GTIFF',
        'compress':'lzw',
        'dtype':'float32',
        'nodata':nodata,
        'blockxsize': 128,
        'blockysize': 128,
        'tiled': True,
        'interleave': 'band'
    }

    ds_name = os.path.splitext(dataset)[0]
    tifs = []
    dates = []
    assets = []

    for bnd in range(bands):
        yr = str(int(nc['time'][bnd]))
        logging.warning(yr)

        dst_name = DATA_DIR + ds_name + '_' + yr + '.tif'
        logging.warning('Destination name: ' + dst_name)

        tifs.append(dst_name)
        dates.append(yr)
        assets.append(getAssetName(name, resolution, yr))

        logging.warning('Assets: ' + str(assets))

        with rio.open(dst_name, 'w', **profile) as dst:
            logging.warning("STARTING BAND")
            logging.warning(dst.profile)
            
            bnd_data = nc_data[bnd,:,:].astype(profile['dtype'])
            dst.write(bnd_data, indexes=1)
#             windows = dst.block_windows()
#             for win_ix, window in windows:
#                 logging.warning(win_ix)
#                 y_min = window[1][0]
#                 y_max = window[1][1]
#                 x_min = window[0][0]
#                 x_max = window[0][1]
#                 bnd_data = nc_data[bnd,y_min:y_max,x_min:x_max].astype(profile['dtype'])
#                 dst.write(bnd_data, indexes=1, window=window)


DEBUG:rasterio.env:Entering env context: <rasterio.env.Env object at 0x10b1da9e8>
DEBUG:rasterio.env:Starting outermost env
DEBUG:rasterio.env:No GDAL environment exists
DEBUG:rasterio.env:New GDAL environment <rasterio._env.GDALEnv object at 0x104454d30> created
DEBUG:rasterio._env:Logging error handler pushed.
DEBUG:rasterio._env:All drivers registered.
DEBUG:rasterio._env:Started GDALEnv <rasterio._env.GDALEnv object at 0x104454d30>.
DEBUG:rasterio.env:Entered env context: <rasterio.env.Env object at 0x10b1da9e8>
DEBUG:rasterio._io:Path: /Users/nathansuberi/Documents/GitHub/ResourceWatchCode/static-data-connectors/soc_073_gridded_gdp/data/GDP_PPP_30arcsec_v2_1990.tif, mode: w, driver: GTIFF
DEBUG:rasterio._io:Option: ('COMPRESS', b'LZW')
DEBUG:rasterio._io:Option: ('BLOCKXSIZE', b'128')
DEBUG:rasterio._io:Option: ('BLOCKYSIZE', b'128')
DEBUG:rasterio._io:Option: ('TILED', b'TRUE')
DEBUG:rasterio._io:Option: ('INTERLEAVE', b'BAND')
DEBUG:rasterio._io:Input CRS: CRS({'init': 'EPSG:432